In [2]:
!pip install lamini
!pip install util

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.0/694.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 14.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement util (from versions: none)
ERROR: No matching distribution found for util


In [3]:
import lamini
lamini.api_key = "9bfb9cf42602648f7ea46db4d2405b0a33f858ca07ce39868b862811919866d9"

llm = lamini.Lamini("meta-llama/Meta-Llama-3.1-8B-Instruct")
print(llm.generate("""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

how llama3 is difer from gpt?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

"""))

You're asking about the differences between LLaMA and GPT!

LLaMA and GPT are both large language models developed by Meta AI, but they have some key differences:

**1. Training data:**
	* GPT (Generative Pre-trained Transformer) was trained on a massive dataset of text from the internet, specifically from the Common Crawl dataset, which includes a wide range of texts from the web.
	* LLaMA (Large Language Model Meta AI) was trained on a dataset that is similar to GPT's, but with a focus on more conversational and dialogue-based data, as well as a larger portion of text from the internet.

**2. Architecture:**
	* GPT is based on the transformer architecture, which is a type of neural network designed for natural language processing tasks.
	* LLaMA is also based on the transformer architecture, but with some modifications to improve its performance on conversational tasks.

**3. Size and complexity:**
	* GPT-3 has 175 billion parameters, making it one of the largest language models in t

In [21]:
import logging
import sqlite3
import pandas as pd

In [36]:
logger = logging.getLogger(__name__)
engine = sqlite3.connect("./nba_roster.db")

In [24]:
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3-8B-Instruct")

In [25]:
# Meta Llama 3 Instruct uses a prompt template, with special tags used to indicate the user query and system prompt.
# You can find the documentation on this [model card](https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/#meta-llama-3-instruct).
def make_llama_3_prompt(user, system=""):
    system_prompt = ""
    if system != "":
        system_prompt = (
            f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
        )
    return f"<|begin_of_text|>{system_prompt}<|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [26]:
def get_schema():
    return """\
0|Team|TEXT
1|NAME|TEXT
2|Jersey|TEXT
3|POS|TEXT
4|AGE|INT
5|HT|TEXT
6|WT|TEXT
7|COLLEGE|TEXT
8|SALARY|TEXT eg.
"""

In [27]:
user = """Who is the highest paid NBA player?"""

In [28]:
system = f"""You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
{get_schema()}

Write a sqlite query to answer the following question. Follow instructions exactly"""

In [29]:
print(system)

You are an NBA analyst with 15 years of experience writing complex SQL queries. Consider the nba_roster table with the following schema:
0|Team|TEXT 
1|NAME|TEXT  
2|Jersey|TEXT 
3|POS|TEXT
4|AGE|INT 
5|HT|TEXT 
6|WT|TEXT 
7|COLLEGE|TEXT 
8|SALARY|TEXT eg. 


Write a sqlite query to answer the following question. Follow instructions exactly


In [30]:
prompt = make_llama_3_prompt(user, system)

In [31]:
print(llm.generate(prompt, max_new_tokens=200))

To answer this question, we can use the following SQLite query:

```sql
SELECT NAME, SALARY
FROM nba_roster
ORDER BY SALARY DESC
LIMIT 1;
```

This query will return the name and salary of the highest paid NBA player. The `ORDER BY SALARY DESC` clause sorts the players by their salary in descending order (highest to lowest), and the `LIMIT 1` clause ensures that we only get the top result.


In [32]:
result = llm.generate(prompt, output_type={"sqlite_query": "str"}, max_new_tokens=200)

In [ ]:
df = pd.read_sql(result['sqlite_query'], con=engine) # it will show the output in the data frame(query result)